In [1]:

# Program to read cube info file and convert html files
# Author mithlesh.patel@corp.badoo.com

import os, fnmatch
import string
import re
import glob
import codecs
import subprocess

ascii = set(string.printable) 

def remove_non_ascii(s):
    return filter(lambda x: x in ascii, s)

def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename

cubeInfoLocation ='bi18.mlan:/local/bi/microstrategy/IntelligenceServer/Cube/'
htmlLocation = '/usr/share/tomcat/webapps/ProjectDocumentation/Cube_Info/'

#cubeInfoLocation ='/Users/mithleshpatel/Documents/ImpDocs/Cube Info/'
#htmlLocation = '/Users/mithleshpatel/Documents/ImpDocs/Cube Info/HTMLFiles/'

test = htmlLocation+'*'
r = glob.glob(test)
for i in r:
    os.remove(i)
    
filecount=0
word='select'
sqlTags=['select','with']
print "Cube info is being processed, kindly wait...\n"
for cubeInfoFile in find_files(cubeInfoLocation, '*_info.cube'):
    #print cubeInfoFile
    html_location_split = cubeInfoFile.split("/")
    html_lenght = len(html_location_split)
    html_file_name = html_location_split[html_lenght-1]
    cubeCacheID = html_file_name.split(".")
    htmlCubeInfoName = cubeCacheID[0].split("_")[0]+'.html'
    htmlFileLocation =  htmlLocation+htmlCubeInfoName

    # Read data from cube info file
    strings = subprocess.check_output(["strings", cubeInfoFile])
    lines=strings.split('\n')
    with open(htmlFileLocation,'wb') as file:
        file.write('<head>')
        file.write('<link rel="icon" href="/ProjectDocumentation/Images/Main.jpg" type="image/gif" sizes="16x16">')
        file.write('</head>')
        file.write('<img src="/ProjectDocumentation/Images/Main.jpg">')
        file.write('<br />')
        file.write('<hr width=100%')
        file.write('<br />')
        if word in strings:
            x=0
            sqlstartIndex=0
            sqlendIndex=0
            for x in range(len(lines)-1):
                for sqlWord in sqlTags:
                    if sqlWord in lines[x]:
                        sqlstartIndex = x
                        #print lines[x]
                        break
                if sqlstartIndex >0:
                    break
                        
            for x in range(len(lines)-1):
                if '[Populate Report Data]' in lines[x]:
                    sqlendIndex=x
                    break
                    
            #print sqlstartIndex,sqlendIndex
            for line in lines[sqlstartIndex-1:sqlendIndex]:
                
                if 'Exasol Production' in line:
                    file.write('<br />')
                    htline='<font face="arial" color="red" size="2">'+line+'</font>'
                    file.write(htline)
                    file.write('<br />')
                elif ('from') in line:
                    htline='<font face="arial" color="blue" size="2">'+line+'</font>'
                    file.write(htline+' ')
                    file.write('<br />')
                elif 'join' in line:
                    htline='<font face="arial" color="blue" size="2">'+line+'</font>'
                    file.write(htline+' ')
                    file.write('<br />')
                elif 'where' in line:
                    htline='<font face="arial" color="blue" size="2">'+line+'</font>'
                    file.write(htline+' ')
                    file.write('<br />')
                else:
                    htline='<font face="arial" color="black" size="2">'+line+'</font>'
                    file.write(htline)
                    file.write('<br />')
                    
        else:
            htlines='<font face="arial" color="black" size="2">'+'No SQL generated for this cube !'+'</font>'
            file.write(htlines)
            file.write('<br />')
            
        file.close()
        filecount+=1
if filecount==0:
    print 'No files processed, please check whether cube info files are in place.'
else:
    print "Cube information has been updated with %d files" %filecount

Cube info is being processed, kindly wait...

No files processed, please check whether cube info files are in place.
